In [1]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

2022-12-20 00:15:46.631454: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 00:15:46.721235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 00:15:46.721253: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-20 00:15:47.283673: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)
df

,zcmb,mb,dmb,errors
0,0.503084,23.001698,0.088031,0.120219
1,0.580724,23.573937,0.090132,0.121435
2,0.494795,22.960139,0.088110,0.089552
3,0.345928,22.398137,0.087263,0.119729
4,0.677662,24.078115,0.098356,0.100088
...,...,...,...,...
735,0.027064,16.504006,0.141685,0.185894
736,0.025468,15.797848,0.143429,0.193666
737,0.023810,15.895501,0.144315,0.184957
738,0.023867,16.068268,0.144350,0.175960


In [3]:
dmag =df["dmb"]

df2=df['errors']+df['dmb']**2

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.75
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((555, 1), (185, 1), (555, 2), (185, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=False)
                                   ]

n_cols = 1

In [7]:
epochs=200

In [8]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
    
    for i in range(hparams['HP_LAYERS']):        
        model.add(Dense(hparams['HP_NUM_UNITS'], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams['HP_LEARNING'], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    train = model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams['HP_BATCHSIZE'], shuffle=False, verbose=False)

    _, loss = model.evaluate(X_test, Y_test)
    
    return model, loss, train.history

# Grid results


In [9]:
# 2	50	0.00100	16
hparams1 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 4	150	0.00010	16
hparams2 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 150, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.0001}
# 3	200	0.00100	16
hparams3 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 4	100	0.00100	16
hparams4 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 3	200	0.00100	16
hparams5 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 4	150	0.00010	16
hparams6 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 150, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.0001}
# 3	200	0.00100	16
hparams7 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 3	200	0.00100	16
hparams8 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}
# 4	100	0.00100	16
hparams9 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.001}

In [12]:
models_grid = []
mse_grid = []
mae_grid = []
r2_grid = []
histories_grid = []
hparams_grid = [hparams1, hparams2, hparams3,hparams4,hparams5,hparams6,hparams7,
                hparams8,hparams9]

for i in range(9):
    model, loss_test, history = train_test_model(hparams_grid[i])
    models_grid.append(model)
    mse_grid.append(loss_test)
    histories_grid.append(history)
    y = model.predict(X_test)
    r2_grid.append(r2_score(y, Y_test))
    mae_grid.append(mean_absolute_error(y, Y_test))


for i in range(9):
    print("Run {}: {:.4f}\t{:.4f}\t{:.4f}".format(i, mse_grid[i], mae_grid[i], r2_grid[i]))

6/6 [==============================] - 0s 5ms/step
Run 0: 0.0370	0.1141	0.6914
Run 1: 0.0361	0.1133	0.5394
Run 2: 0.0350	0.1093	0.4285
Run 3: 0.0448	0.1222	0.4994
Run 4: 0.0807	0.1714	0.4531
Run 5: 0.0409	0.1189	0.6572
Run 6: 0.0678	0.1478	0.6657
Run 7: 0.0369	0.1101	0.4174
Run 8: 0.0340	0.1035	0.6679


# Genetic results

## A

In [9]:
# 4	50	0.00010	2
hparams_gena1 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 2	200	0.00010	2
hparams_gena2 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 2	100	0.00010	2
hparams_gena3 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 4	150	0.00010	4
hparams_gena4 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 150, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 3	100	0.00010	2
hparams_gena5 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 2	100	0.00010	2
hparams_gena6 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 1	200	0.00100	4
# 1 	50 	0.0010 	2
hparams_gena7 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.001}
# 2	200	0.00010	2
hparams_gena8 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 2	200	0.00100	8
hparams_gena9 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 8, 'HP_LEARNING':0.0001}

In [10]:
models_gena = []
mse_gena = []
mae_gena = []
r2_gena = []
histories_gena = []
hparams_gena = [hparams_gena1, hparams_gena2, hparams_gena3,hparams_gena4,hparams_gena5,hparams_gena6,hparams_gena7,
                hparams_gena8,hparams_gena9]

for i in range(9):
    model, loss_test, history = train_test_model(hparams_gena[i])
    models_gena.append(model)
    mse_gena.append(loss_test)
    histories_gena.append(history)
    y = model.predict(X_test)
    r2_gena.append(r2_score(y, Y_test))
    mae_gena.append(mean_absolute_error(y, Y_test))
    
for i in range(9):
    print("Run {}: {:.4f}\t{:.4f}\t{:.4f}".format(i, mse_gena[i], mae_gena[i], r2_gena[i]))

2022-12-20 00:16:37.861435: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 00:16:37.861697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-20 00:16:37.861783: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-20 00:16:37.861855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-20 00:16:37.861926: W tensorflow/c

6/6 [==============================] - 0s 737us/step
Run 0: 0.0363	0.1069	0.5894
Run 1: 0.0315	0.0996	0.6893
Run 2: 0.0313	0.0997	0.6866
Run 3: 0.0340	0.1049	0.7091
Run 4: 0.0363	0.1070	0.6357
Run 5: 0.0316	0.0998	0.6515
Run 6: 0.0323	0.1023	0.4249
Run 7: 0.0316	0.0997	0.6787
Run 8: 0.0411	0.1157	0.6956


## B

In [11]:
# 4	200	0.00010	2
hparams_genb1 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 4	150	0.00010	2
hparams_genb2 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 150, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 3	100	0.00010	4
hparams_genb3 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 4	100	0.00100	4
hparams_genb4 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 4	100	0.00010	4
hparams_genb5 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 3	100	0.00010	4
hparams_genb6 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 3	200	0.00100	16
hparams_genb7 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 16, 'HP_LEARNING':0.0001}
# 4	50	0.00010	4
hparams_genb8 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 2	100	0.00010	2
hparams_genb9 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}

In [12]:
models_genb = []
mse_genb = []
mae_genb = []
r2_genb = []
histories_genb = []
hparams_genb = [hparams_genb1, hparams_genb2, hparams_genb3,hparams_genb4,hparams_genb5,hparams_genb6,hparams_genb7,
                hparams_genb8,hparams_genb9]

for i in range(9):
    model, loss_test, history = train_test_model(hparams_genb[i])
    models_genb.append(model)
    mse_genb.append(loss_test)
    histories_genb.append(history)
    y = model.predict(X_test)
    r2_genb.append(r2_score(y, Y_test))
    mae_genb.append(mean_absolute_error(y, Y_test))
    
for i in range(9):
    print("Run {}: {:.4f}\t{:.4f}\t{:.4f}".format(i, mse_genb[i], mae_genb[i], r2_genb[i]))

6/6 [==============================] - 0s 699us/step
Run 0: 0.0470	0.1285	0.7054
Run 1: 0.0407	0.1177	0.6659
Run 2: 0.0333	0.1009	0.6912
Run 3: 0.0515	0.1449	0.0920
Run 4: 0.0342	0.1075	0.6933
Run 5: 0.0343	0.1029	0.6792
Run 6: 0.0386	0.1102	0.6795
Run 7: 0.0333	0.1021	0.7204
Run 8: 0.0305	0.0982	0.6779


## C

In [13]:
# 1	200	0.00100	4
hparams_genc1 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 4	50	0.00010	4
hparams_genc2 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 50, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 2	100	0.00010	2
hparams_genc3 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 2	200	0.00100	8
hparams_genc4 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 8, 'HP_LEARNING':0.001}
# 1	200	0.00100	4
hparams_genc5 = {'HP_LAYERS': 1, 'HP_NUM_UNITS': 200, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.001}
# 4	100	0.00010	4
hparams_genc6 = {'HP_LAYERS': 4, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 2	150	0.00010	2
hparams_genc7 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 150, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}
# 3	100	0.00010	4
hparams_genc8 = {'HP_LAYERS': 3, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 4, 'HP_LEARNING':0.0001}
# 2	100	0.00010	2
hparams_genc9 = {'HP_LAYERS': 2, 'HP_NUM_UNITS': 100, 'HP_BATCHSIZE': 2, 'HP_LEARNING':0.0001}

In [14]:
models_genc = []
mse_genc = []
mae_genc = []
r2_genc = []
histories_genc = []
hparams_genc = [hparams_genc1, hparams_genc2, hparams_genc3,hparams_genc4,hparams_genc5,hparams_genc6,hparams_genc7,
                hparams_genc8,hparams_genc9]

for i in range(9):
    model, loss_test, history = train_test_model(hparams_genc[i])
    models_genc.append(model)
    mse_genc.append(loss_test)
    histories_genc.append(history)
    y = model.predict(X_test)
    r2_genc.append(r2_score(y, Y_test))
    mae_genc.append(mean_absolute_error(y, Y_test))

for i in range(10):
    print("Run {}: {:.4f}\t{:.4f}\t{:.4f}".format(i, mse_genc[i], mae_genc[i], r2_genc[i]))

6/6 [==============================] - 0s 704us/step
Run 0: 0.0339	0.1043	0.3435
Run 1: 0.0330	0.1021	0.4566
Run 2: 0.0314	0.0980	0.6970
Run 3: 0.0357	0.1120	0.5074
Run 4: 0.0319	0.1015	0.3628
Run 5: 0.0328	0.1039	0.6673
Run 6: 0.0317	0.0989	0.6083
Run 7: 0.0329	0.1022	0.6479
Run 8: 0.0338	0.1006	0.6747
